In [1]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding

In [2]:
import pandas as pd
import re
df = pd.read_csv("/home/miza/magisterka/Magisterka/src/data/bert.csv")
def preprocess_text(text):
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', text.strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    text = re.sub(r'\s*\b[a-zA-Z]\b\s*', ' ', text).strip()
    return text

def preprocess_cost(text):
    text = text.lower() 
    text=text.strip()  
    #text = re.sub(r'\s+', '', text)
    pattern = re.compile(r'(\d{1,3}\.\d{2})\d?')
    return pattern.findall(text)

def clean_token(token):
    for t in token: 
        if len(t) == 1:
            print(t)
            token.remove(t)
    return token
    

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_2347/2118193583.py:7: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)
/tmp/ipykernel_2347/2118193583.py:7: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)


FileNotFoundError: [Errno 2] No such file or directory: '/home/miza/magisterka/Magisterka/src/data/bert.csv'

In [ ]:
import transformers
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('dkleczek/bert-base-polish-uncased-v1')
bert_model = BertModel.from_pretrained('dkleczek/bert-base-polish-uncased-v1')


In [ ]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

def data_preprocess(df):
    df['preprocessed_text'] = df['OCR_product'].apply(lambda x: preprocess_text(x))
    df['cost'] = df['OCR_product'].apply(lambda x: preprocess_cost(x))
    embedings = calculate_embedings(df['preprocessed_text'].tolist())
    le=LabelEncoder()
    categories =  le.fit_transform(df['Category'])
    return df, embedings, categories

def calculate_embedings(df):
    inputs = tokenizer(df, return_tensors='pt', truncation=True, max_length=32, padding='max_length')
    with torch.no_grad():
        outputs = bert_model(**inputs)
        #cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return outputs.pooler_output

In [ ]:
df = pd.read_csv("/home/miza/Magisterka/src/data/annotations/annotations.csv")
df, embedings, categories = data_preprocess(df)


In [ ]:
X, Y = embedings, categories
X_test, X_train, Y_test, Y_train = train_test_split(X,Y,test_size=0.7, random_state=42)

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()

In [ ]:
model.fit(X_train,Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [2]:
preds = model.predict(X_test)
print(classification_report(Y_test, preds))

NameError: name 'model' is not defined

In [ ]:
df_val = pd.read_csv("/home/miza/Magisterka/src/data/annotations/annotations.csv")
df_val, embedings_val, categories_val = data_preprocess(df_val)


In [251]:
preds_val = model.predict(embedings_val)

In [253]:
print(classification_report(categories_val,preds_val))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.18      0.80      0.30         5
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         7
           4       0.00      0.00      0.00         4
           5       0.20      0.25      0.22         4
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         5

    accuracy                           0.12        40
   macro avg       0.04      0.11      0.05        40
weighted avg       0.04      0.12      0.06        40



/home/miza/magisterka/Magisterka/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miza/magisterka/Magisterka/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miza/magisterka/Magisterka/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr